## Text Analytics Group Assignment 2
### ArjunVarma_AnyingLi_YuanzhuoPeng_JiaqiuWang_NimishAmlathe

In [1]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
yelp = pd.read_csv('Yelp Data Restaurant Reviews Ratings.csv')
yelp['rating'] = yelp['stars'].map(lambda x: 'High' if x >3  else 'Low')
yelp[:5]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,Review,rating
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,This location is out of business. I drove by i...,Low
1,2,2,2,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,= = = = = = CLOSED = = = = = =This JB s locati...,Low
2,4,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,This is just a basic (albeit mini) chain greas...,High
3,3,0,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Whenever I offer to take my mom out to lunch s...,Low
4,3,7,9,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,If I say it wasn t as bad as I was expecting i...,Low


In [3]:
yelp.columns.values

array(['stars', 'votes_cool', 'votes_funny', 'votes_useful', 'Cheap',
       'Moderate', 'Expensive', 'VeryExpensive', 'American', 'Chinese',
       'French', 'Japanese', 'Indian', 'Italian', 'Greek', 'Mediterranean',
       'Mexican', 'Thai', 'Vietnamese', 'Others', 'Review', 'rating'], dtype=object)

#### Task A. Ignore the text (reviews) and run a classification model with the numeric data (you can use standard methods like logistic regression, k-nearest neighbors or anything else). What is the best accuracy of your model with numeric data?

In [4]:
# Take only a random subset of 2000 rows
yelp_subset = yelp.sample(frac=.1,random_state=5)
yelp_subset = yelp_subset.reset_index()
yelp_subset = yelp_subset.drop('index',axis=1)
yelp_subset[:3]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,Review,rating
0,5,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,I ll be honest...I m not a food connoisseur b...,High
1,2,0,1,3,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,Made reservations for 5:30 PM on a Saturday ni...,Low
2,5,0,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,This place has an awesome Happy Hour. The envi...,High


In [5]:
#split train and test data
from sklearn.cross_validation import StratifiedShuffleSplit 

y = yelp_subset['rating'].map(lambda y: 1 if y == 'High'  else 0)

X = yelp_subset.drop(['stars','Review','rating'], axis=1)

Review = yelp_subset['Review']

sss = StratifiedShuffleSplit(y, test_size=0.3, random_state=1)
for train_index, test_index in sss:
    break

train_x, train_y,train_review = X.values[train_index],y.values[train_index],Review[train_index]
test_x, test_y, test_review = X.values[test_index], y.values[test_index],Review[test_index]

In [6]:
#knn model with numerical inputs only
from sklearn import neighbors
modelknn = neighbors.KNeighborsClassifier(n_neighbors=15, weights='uniform', p=2)
modelknn.fit(train_x,train_y)

from sklearn import metrics
prediction_on_training = modelknn.predict(train_x)
print "Train set accuracy score:", metrics.accuracy_score(train_y, prediction_on_training)

predicted_classes = modelknn.predict(test_x)
print "Test set accuracy score:", metrics.accuracy_score(test_y, predicted_classes)

Train set accuracy score: 0.695714285714
Test set accuracy score: 0.703333333333


In [7]:
#logistic regression with numerical inputs only
from sklearn.linear_model import LogisticRegression
modellog = LogisticRegression()
modellog.fit(train_x, train_y)

from sklearn import metrics
prediction_on_training = modellog.predict(train_x)
print "Train set accuracy score:", metrics.accuracy_score(train_y, prediction_on_training)

predicted_classes = modellog.predict(test_x)
print "Test set accuracy score:", metrics.accuracy_score(test_y, predicted_classes)

Train set accuracy score: 0.692857142857
Test set accuracy score: 0.701666666667


In [8]:
#naive bayes with numerical inputs only
from sklearn import naive_bayes
modelnby = naive_bayes.MultinomialNB()
modelnby.fit(train_x, train_y)

from sklearn import metrics
prediction_on_training = modelnby.predict(train_x)
print "Train set accuracy score:", metrics.accuracy_score(train_y, prediction_on_training)

predicted_classes = modelnby.predict(test_x)
print "Test set accuracy score:", metrics.accuracy_score(test_y, predicted_classes)

Train set accuracy score: 0.687142857143
Test set accuracy score: 0.686666666667


So the best accuracy with using only numeric data for prediction is about 70.3% on test data with k nearest neighbor model.

#### Task B. Perform a supervised classification on a subset of the corpus using the reviews only. You can write your code in Python or R. What accuracy do you get from this text mining exercise?

In [9]:
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


import re
import string
import nltk
from nltk import word_tokenize          
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords


stemmer = PorterStemmer()
wnl = WordNetLemmatizer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def lemmatize(tokens,wnl):
    lem=[]
    for item in tokens:
        lem.append(wnl.lemmatize(item))
    return lem

def my_tokenizer(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    text = re.sub('[0-9]+','',text)
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    #result = stem_tokens(tokens, stemmer)
    #result = lemmatize(tokens, wnl)
    result = tokens
    return result
    

vectorizer = TfidfVectorizer(min_df=3,max_df=0.9,tokenizer=my_tokenizer,
 ngram_range=(1, 2), 
 stop_words='english',
 strip_accents='unicode',max_features=200)

text_1 = vectorizer.fit_transform(train_review)

In [10]:
from nltk import NaiveBayesClassifier
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

text_2 = vectorizer.transform(test_review)

nb =  MultinomialNB().fit(text_1, train_y)

y_nb_predicted = nb.predict(text_2)


print "MODEL: Multinomial Naive Bayes\n"

print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, y_nb_predicted))

MODEL: Multinomial Naive Bayes

The accuracy for this classifier is 0.72


So the accuracy with using only review data for prediction is about 72% on test data with Multinomial Naive Bayes Model.

#### Task C. Combine the numeric data and the text classification model (in task B) to create a “hybrid” model. It is your task to figure out how to do this. Now run this hybrid classification model and compare the results with those in A and B. Does the numeric data add to the predictive power relative to text?

In [11]:
from sklearn.naive_bayes import MultinomialNB
from operator import itemgetter

N = 50
vocabulary = np.array([t for t, i in sorted(vectorizer.vocabulary_.iteritems(), key=itemgetter(1))])

topN = np.argsort(nb.coef_[0])[-N:]

print "\nThe top %d most informative features: \n%s" % (N, ','.join(vocabulary[topN]))


The top 50 most informative features: 
perfect,times,hour,meal,people,rice,right,wait,tacos,mexican,got,sauce,salad,definitely,order,burger,dinner,eat,atmosphere,excellent,cheese,staff,bar,favorite,ordered,phoenix,happy,pizza,awesome,friendly,lunch,try,fresh,little,nice,amazing,restaurant,menu,delicious,chicken,time,really,best,love,like,service,place,good,food,great


In [12]:
most_informative = vocabulary[topN]

for w in most_informative:
    X[w]= 0

def create_features(tokens,i):
    document_words = set(tokens)
    features = {}
    for word in most_informative:
        if word in document_words:
            X.loc[i,word] = 1
        
X.head()

,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,French,...,time,really,best,love,like,service,place,good,food,great
0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,3,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
for i in range(1000):
    text = "".join([ch for ch in Review[i] if ch not in string.punctuation])
    text = re.sub('[0-9]+','',text)
    tokens = nltk.word_tokenize(text.lower().decode('utf-8'))
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    feature = create_features(tokens,i)

X.head()

,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,French,...,time,really,best,love,like,service,place,good,food,great
0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,1,0
1,0,1,3,0,0,1,0,1,0,0,...,1,0,1,0,1,1,1,1,1,1
2,0,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,1,1,0
3,0,0,0,0,0,0,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
nb_text =  MultinomialNB().fit(X.values[train_index], train_y)

text_predicted = nb_text.predict(X.values[test_index])


print "Hybrid MODEL: Multinomial Naive Bayes\n"

print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, text_predicted))

Hybrid MODEL: Multinomial Naive Bayes

The accuracy for this classifier is 0.723333333333


In [15]:
sample_weight = np.random.rand(y.shape[0])

sw_train, sw_test = sample_weight[train_index], sample_weight[test_index]

from sklearn.calibration import CalibratedClassifierCV

from sklearn.metrics import brier_score_loss


clf = MultinomialNB()
model_original = clf.fit(X.values[train_index], train_y)  # GaussianNB itself does not support sample-weights
prob_pos_clf = clf.predict_proba(X.values[test_index])[:, 1]
clf_predicted =model_original.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, clf_predicted))

# Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(clf, cv=10, method='isotonic')
model_isotonic = clf_isotonic.fit(X.values[train_index], train_y, sw_train)
prob_pos_isotonic = clf_isotonic.predict_proba(X.values[test_index])[:, 1]
isotonic_predicted = model_isotonic.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, isotonic_predicted))

# Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(clf, cv=10, method='sigmoid')
model_sigmoid  = clf_sigmoid.fit(X.values[train_index], train_y, sw_train)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X.values[test_index])[:, 1]
sigmoid_predicted = model_sigmoid.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, sigmoid_predicted))

print("Brier scores: (the smaller the better)")

clf_score = brier_score_loss(test_y, prob_pos_clf, sw_test)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(test_y, prob_pos_isotonic, sw_test)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(test_y, prob_pos_sigmoid, sw_test)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

The accuracy for this classifier is 0.723333333333
The accuracy for this classifier is 0.716666666667
The accuracy for this classifier is 0.706666666667
Brier scores: (the smaller the better)
No calibration: 0.196
With isotonic calibration: 0.196
With sigmoid calibration: 0.200


#### Task D. Use unsupervised sentiment analysis on the reviews (with SentiStrength or any other tool) and use the sentiment scores to predict high/low rating. Compare and contrast the results of tasks B and D. What can you conclude from your analysis?

In [46]:
# Export yelp_subset
yelp_subset.to_csv('yelp_subset.csv',index=False)

# Run sentistrength on column 21 of yelp_subset to get positive and negative sentiments of each review
# Output from sentistrength: yelp_subset_senti+results.txt

# Load back into python
yelp_senti = pd.read_table('yelp_subset_senti+results.txt')
yelp_senti['Overall_senti'] = yelp_senti['Postive_senti'] + yelp_senti['Negative_senti']

# Get predicted ratings
yelp_senti["Pred_rating"] = np.where((yelp_senti['Overall_senti'] > 0),"High","Low")
yelp_senti[:3]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Mexican,Thai,Vietnamese,Others,Review,Rating_final,Postive_senti,Negative_senti,Overall_senti,Pred_rating
0,2,0,1,3,0,0,1,0,1,0,...,0,0,0,0,Made reservations for 5:30 PM on a Saturday ni...,Low,3,-4,-1,Low
1,5,0,1,1,0,1,0,0,1,0,...,0,0,0,0,This place has an awesome Happy Hour. The envi...,High,3,-1,2,High
2,3,0,0,0,0,0,0,1,1,0,...,0,0,0,0,Noted on the OT reservation it was hubby s 50t...,Low,4,-3,1,High


In [48]:
# Checking accuracy
from sklearn.metrics import confusion_matrix
y_true = yelp_senti['Rating_final'].tolist()
y_pred = yelp_senti['Pred_rating'].tolist()
cm_sk = confusion_matrix(y_true, y_pred)

cm = nltk.ConfusionMatrix(y_true, y_pred)
print (cm.pretty_format(sort_by_count=True, show_percents=True))
print 'The accuracy is:',round(float(cm_sk[1][1]+cm_sk[0][0])/float(sum(cm_sk)),3)

     |      H        |
     |      i      L |
     |      g      o |
     |      h      w |
-----+---------------+
High | <54.1%> 14.6% |
 Low |  14.2% <17.2%>|
-----+---------------+
(row = reference; col = test)

The accuracy is: 0.712


#### Task E. Implement the PMI approach to sentiment analysis (in either Python or R), and run the classification model with the sentiment scores. How do your results compare with those in Task D?

In [37]:
import functools
from nltk.corpus import stopwords as stp
from nltk import pos_tag
import requests
from bs4 import BeautifulSoup

In [31]:
def features_extraction(text, rule_1, rule_2, no_match):
    
    wc = []
    append = wc.append
    text = text.lower()
    tokens = re.split("\W+", text)
    stopwords = stp.words("english")
    remove = tokens.remove
    while "" in tokens:
        remove("")
    for token in tokens:
        if token in stopwords:
            remove(token)
    token_pos = pos_tag(tokens)
    
    for i in xrange(len(token_pos)-1):
        if (token_pos[i][1], token_pos[i+1][1]) not in rule_1 and\
            (token_pos[i][1], token_pos[i+1][1]) not in rule_2:
                continue
        elif (token_pos[i][1], token_pos[i+1][1]) in rule_1:
            append((token_pos[i][0], token_pos[i+1][0]))
        elif (token_pos[i][1], token_pos[i+1][1]) in rule_2:
            try:
                if token_pos[i+2] not in no_match:
                    append((token_pos[i][0], token_pos[i+1][0]))
            except IndexError:
                append((token_pos[i][0], token_pos[i+1][0]))
        else:
            continue

    return wc

In [32]:
def Semantic_Orientation(phrases,positive="excellent", negative="poor", prior = 1,engine="google",distance=10,threshold=None):
  
    so_positive = 0.01
    so_negative = 0.01
    so_avg = 0
    get = requests.get

    engine.lower() == "google"
    url = "http://www.google.com/search?q=%s"
    id_pattern = "resultStats"
    
    for phrase in phrases:
        term = "%22{}+{}%22+AROUND({})+%22{}%22".format(phrase[0], distance, phrase[1], positive)
        page = get(url % term)
        soup = BeautifulSoup(page.text, "lxml")
        try:
            rtr_pos = int("".join(re.split("\D+",soup.find("div", id=id_pattern).get_text().encode("utf-８"))))
        except AttributeError:
            rtr_pos = 0
            
        # if rtr_pos == None:
        #     rtr_pos = 0
        
        term = "%22{}+{}%22+AROUND({})+%22{}%22".format(phrase[0], distance, phrase[1], negative)
        page = get(url % term)
        soup = BeautifulSoup(page.text, "lxml")
        try:
            rtr_neg = int("".join(re.split("\D+",soup.find("div", id=id_pattern).get_text().encode("utf-８"))))
        except AttributeError:
            rtr_neg = 0
            
        # if rtr_neg == None:
         #   rtr_neg =0
            
        so_positive += rtr_pos
        so_negative += rtr_neg
        
        so_avg += math.log(so_positive/so_negative/prior)
    
    #if len(phrases)==0:
     #   so_avg=0
    #else:
        so_avg = so_avg*1.0/len(phrases)
    
    if threshold == None:
        return so_avg
    else:
        return int(so_avg>threshold)

In [40]:
# Defining Patters for Feature Extraction
rule_1 = [("JJ", "NN"), ("JJ", "NNS"), ("RB", "VB"), ("RB", "VBD"), ("RB", "VBN"), ("RB", "VBG"),("RBR", "VB"), ("RBR", "VBD"), ("RBR", "VBN"), ("RBR", "VBG"),
          ("RBS", "VB"), ("RBS", "VBD"), ("RBS", "VBN"), ("RBS", "VBG")]
rule_2 = [("RB", "JJ"), ("RBR", "JJ"), ("RBS", "JJ"),("JJ", "JJ"),("NN", "JJ"), ("NNS", "JJ")]
no_match = ["NN", "NNS"]

# Extracting Features

features_extract = functools.partial(features_extraction, rule_1=rule_1, rule_2=rule_2, no_match=no_match)
features=yelp_subset.Review.map(features_extract)

In [41]:
# Calling the function that gives Semantic Orientation for a review.

Opinion_Orientation = functools.partial(Semantic_Orientation,positive="excellent", negative="poor",prior = 1,engine="google",distance=5,threshold=None)

# Mapping the function to our review list(sample :12)

prediction = features[:12].map(Opinion_Orientation)

ValueError: invalid literal for int() with base 10: ''

In [44]:
# Checking accuracy
y_pred = prediction.tolist()
y_true = y[:12].tolist() 


cm_sk = confusion_matrix(y_true, y_pred)
cm = nltk.ConfusionMatrix(y_true, y_pred)


print (cm.pretty_format(sort_by_count=True, show_percents=True))
if cm_sk[1][1] ==None:
    sum_sk[1][1]=0;
print 'The accuracy is:',round(float(cm_sk[1][1]+cm_sk[0][0])/float(len(y_pred)),3)

NameError: name 'prediction' is not defined

#### Task F. What are the top 5 “attributes” of a restaurant that are associated with (i) high and (ii) low ratings? That is, when people rate a restaurant high or low, are they more likely to mention service, ambiance, etc.? 

In [45]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans

In [66]:
pos_reviews  = yelp.ix[yelp.rating == 'High']['Review']
vectorizer = CountVectorizer(min_df = 0, stop_words = 'english')
dtm = vectorizer.fit_transform(pos_reviews)

In [67]:
#Perform Latent Semantic Analysis on the matrix with 5 components
lsa = TruncatedSVD(5, algorithm = 'randomized')
dtm_lsa = lsa.fit_transform(dtm)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
pos_lsa = pd.DataFrame(dtm_lsa, index = pos_reviews, columns = ["component_1","component_2","component_3","component_4","component_5" ])

##### The reviews are ranked based on the score in individual components, then topics are extracted from the top reviews.

component 1: people tend to talk about the service

In [68]:
pos_lsa.sort_values('component_1', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
absolutely delicious indian fare. can even get completely vegan. it s small inside but they turn tables quick so you shouldn t have to wait long. for our own food we waited only a few minutes before we had piping hot tasty curry on our table. we had the chickpea ceviche (recommended by our server) for an app and it was so fresh and cool and crisp tasting. then onto the bombay plate 5 picks of entrees with lettuce wraps and each thing we had was sooooo good....spinach and tofu vegetable curry eggplant curry vegetable makki and lentils. came with a little roti bread and we tore into it. for desert we could only the sorbet since the other options had allergy inducing pistachios but it was a nice cool rich tasting treat. our server was severely genuine and sincere and made us feel like valued customers. of course she was from alabama so we would ve expected nothing less. a pleasure to have her taking care of us. a great place and i know we ll be back here. 30 bucks for two with 2 large apps a large entree and desert.,0.998425,-0.019216,-0.038637,-0.029912,-0.019772
Oh man am I the only one who loves Macayo s?? They have great chips that are always served extremely quickly and delicious salsa that they will readily fill back up. Whenever my coworker and I feel like eating fatty unhealthy food we always go there for the Baja Burro - oh my gosh...so good! It is a burrito filled with either chicken steak or shrimp covered in baja sauce and cheese. Its freaking delicious! You can also get a bean and cheese burrito Baja Style if you choose and they will add all of this deliciousness to your bean and cheeser.I want to go for happy hour and try their new watermelon and honeydew margs because those sound delish. Anyway I totally recommend this place for a quick lunch I ve never been disappointed and never left even remotely hungry. In fact most times I feel I need a wheelchair.,0.996507,-0.070234,-0.024420,0.027169,0.026563
This little place does a great job pumping out fresh Chinese food. It won t really blow you through the wall into the Avis rental office next door but it s above average. Thus far we ve tried several of the chicken and veggie dishes--they are all very good. The Garlic chicken and Mongolian chicken are favorites. Pei Wok makes everything fresh to order. There are a few clean small tables you could sit at to eat (I think two two-tops and one four-top) but I always get it to go.There are smaller lunch combos for a very cheap prices--cheaper even than going to Panda Express which is relative garbage. For dinner you can get a Large which is just white rice and entree or you can get the Combo which is fried rice entree egg roll and cream cheese wontons. Look maybe a better semantic strategy was in order but they do what they can.The guys running the joint are also very nice and friendly. They always banter with me about my tattoos the weather or anything else that comes up.,0.996463,-0.004482,-0.006548,0.075198,-0.036665
The Windsor drips with attitude and atmosphere. Imagine a really cool gastro pub with high quality (and deliciously savory) food items surrounded in plaid patterns retro-chic (male heavy) influenced art design and a kick-ass patio area. The bar inside offers just about anything your alcoholic heart might desire- with a killer happy hour of $5 beer pitchers (great selection of brews to boot) glasses of wine from the tap (yes I said tapped wine but it s really good stuff) and other delicacies. The staff is extremely friendly- they know when to check in and when to just let your group hang out and enjoy the Windsor. If you re not hung up on meat please try the burger Windsor style- it doesn t get any better for the $11 price- and includes one of their indulgent sides (mac n cheese for the win). Expect a crowd because the establishment is top notch- the owners know a thing or two about creating an experience as they also manage the Postino c

component 2: people like to talk about the food quality

In [69]:
pos_lsa.sort_values('component_2', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Zupa di pesch was phenom!,0.406261,0.708828,-0.460554,0.094817,-0.333787
YUM if you like a pizza with no sauce get the Snow White! Lots of garlic cheese seasoning. Declicious!!,0.634075,0.700003,-0.225270,0.084178,0.223858
Hang on...gotta wipe the cheese off my chin. Oh crap the enchilada sauce is on my keyboard.Cheese crisp with Guacamole. Two cheese enchiladas one for tomorrow.There s a mess. Everywhere. Ahhhhhhhhhhhhhyessssssssssss!,0.609345,0.684888,-0.236577,0.038742,-0.319620
Butterfish and chocolate souffle are pure heaven!,0.459061,0.682092,-0.285400,-0.064395,-0.488277
what is there to be said that hasn t already been?try their sea salt choc chip cookies. with coffee or milk. get all the FREE toppings on your pizza that you like! (egg? garlic? chilies!??!?!) my fave? the roasted corn with goat cheese. put an egg on it!,0.668425,0.674634,-0.250695,0.065184,0.176014
I ve been here dozens and DOZENS of times over the last 25 years. 5 stars without a doubt.Standouts are the Italian beef with lots o hot . Their cheese fries are absolute perfection... here s why. Put fries in a basket smother them with cheese sauce put another hand full of fries in basket SMOTHER AGAIN WITH CHEESE SAUCE.When you are done with these cheese fries you are looking for more fries for the cheese not more cheese for the fries.Then when you walk out school girls and boys break into song and birds start chirping.Oh I have to mention the cheese steak. I don t like the standard way they do it (beef onions cheese) BUT if you ask for it Philly Style they do it with their cheese sauce that they add extra cheddar cheese to.Yum yum yum.,0.693850,0.673816,-0.243373,-0.012165,-0.071864
Skip the full-sized burger and order sliders; my fiance was eyeing mine jealously. Added crispy-not-oily onion rings finished with a chunky banana milkshake and left smiling full but without any greasy spoon remorse.,0.667710,0.672227,-0.287045,-0.092147,0.106719
Wow..got a $5.00 for just for checking in..,0.682862,0.663885,-0.280125,0.038975,0.113877
Pizza is spicy and sweet. Lovely,0.621319,0.654324,-0.274444,-0.287963,-0.166074


component 3: people give have a good overall impression

In [71]:
pos_lsa.sort_values('component_3', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
So good,0.337852,0.112690,0.894676,-0.228117,-0.143785
Very good,0.337852,0.112690,0.894676,-0.228117,-0.143785
ridiculously good ...,0.340530,0.113996,0.894337,-0.225026,-0.143417
Good and unique.,0.351827,0.111250,0.889490,-0.227309,-0.144844
Good tofu!,0.349333,0.117313,0.889022,-0.224999,-0.152380
Good gyro meat. Tzatziki is good.,0.360123,0.127482,0.886213,-0.221036,-0.140814
Never been to the sky harbor location . The ones in CA are real good tho!,0.396344,0.105429,0.880571,-0.193839,-0.137177
I had Filet Mignon as usual and it was not as good as Ruth s Chris or even Sweetwater Traven in VA.,0.360775,0.131982,0.880121,-0.232126,-0.154681
I recommend the Pollo Siciliano and the Pollo basil.Good Stuff.,0.385702,0.109531,0.879443,-0.219770,-0.132355


component 4: a lot of mexican food

In [72]:
pos_lsa.sort_values('component_4', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Chiptole.... enough saidgreat mexican for being americanized chain....massive burrito for $8,0.344643,-0.364541,0.139623,0.802529,0.291177
This is how Mexican food should be served:It should cost about five bucks. You start with some form of tortilla add refried beans spanish rice salsa and lettuce insert cheesey-messy-possibly-meaty-goodness and throw it all in a styrofoam container. Then you ravage the contents of the container until there are fork scratch marks in the styrofoam. Thank you Carolina s for being one of the few the brave and the proud Mexican places to carry out this process with charm and elegance.,0.477516,-0.389773,0.091255,0.778591,0.074328
TOR.TI.LLAS......Delicioso :)!! Andale mi amgos.....,0.547817,-0.281677,0.038531,0.773673,0.143181
spicy new mexican! yum!,0.598091,-0.167737,-0.058211,0.768364,0.142760
The fluffiest rice I have ever eaten!,0.617523,0.176749,-0.009683,0.765245,-0.041608
Hushpuppies and gumbo!,0.468650,-0.041797,0.150965,0.755563,-0.430063
One of our favorite Mexican food restaurants in the valley. Stay away from the Americanized dishes; instead go for the corn and cabbage enchiladas with while sauce egg noodles covered in green chili and assorted tamales.,0.574452,-0.323384,0.009443,0.749321,-0.062106
I Heart Los!!!This is my absolute favorite Mexican Joint!! Lite A with chicken and a pepsi.....mmmmmm heaven.,0.646511,-0.021358,-0.139500,0.747998,0.051047
Ben B. and I visited Al-Hana after a tiresome IKEA outing and BABA-GODDAMN this stuff is YUMMY AS HELL. We nearly stuffed ourselves to capacity on the smoky babaganoush and hummus. We had the chicken shawarma and Ben had something with spiced beef in it... the name lasted on my tongue about as long as the food can t remember it so sorry.Nothing over 3.99. Seriously. You must.,0.568055,-0.353513,0.041101,0.737383,-0.083188


component 5: The overall place, such as atmosphere

In [73]:
pos_lsa.sort_values('component_5', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Everyone I have told about this place is like wtf is a torta. I have to bring them because it can not be described it has to be experienced.,0.468238,-0.049067,-0.119314,-0.221381,0.845636
I would like to know what happen to Coffee Dans restruant I heard it burnt down when did this take place. I worked there as a teenager in 1963. My email is countryman545@yahoo.com.,0.530397,0.042457,-0.159036,-0.204756,0.806014
May look like a dump I a bad neighborhood but this place is always packed at lunch. Hands down my favorite Mexican joint north of the border,0.569175,-0.103344,-0.122710,-0.080772,0.802358
The absolute best!! I love this place and have been coming here for something like 15 years even when I lived in other states (like MA and WA and CA).,0.560613,0.043147,-0.191521,-0.073261,0.801127
Oye que tal mis amigos? This is the premier place if you want to find out what beef chimichangas should taste like. The place has all the charm of an abandoned warehouse but the eats are second-to-none. Take some tortillas home you ll be glad you did!Been eating at this place faithfully for the past five years and the quality has always been consistent. I get it to go but sat in a few times to eat as well. South Phoenix and close to Van Buren what s not to like?,0.538186,-0.085427,-0.146785,-0.207487,0.799038
Bomba! Wow.. I m here eating while I draft this review. Don t sleep on this place!!!,0.431159,-0.227928,-0.109053,-0.359530,0.788033
Sometimes you have to explore and try those hole in the places. Sometimes you get junk and other times if your lucky. You get a place like this. I was astounded yes I said it ASTOUNDED. By how authentic and delicious this place is. If you haven t come here you haven t had real Mexican seafood. Trust me you have to come here. Chase out,0.577348,-0.144812,-0.143110,-0.174413,0.771232
are you joking? Where else can you get the best salami ham any cold cuts anywhere? I just moved here 6 months ago and have been searching for a place like this. Their meats are fantastic. I discovered this place after driving by it so many times. thought I d take a chance. Location is awful. but worth the trip.,0.554736,-0.097604,-0.165878,-0.250172,0.769831
Trust the positive reviews... This place is like no other. When I travel for work to Phoenix this is the first place I think about whether I get there or not. I ve been going here since 1988.During the right time of the year they fly in stone crabs. You are not a true Arizonian if you haven t been here..... What an institution.... I wish I had this place in Los Angeles.....,0.515281,-0.161290,-0.150112,-0.306597,0.769374


##### perform the same analysis on negative reviews

In [74]:
neg_reviews  = yelp.ix[yelp.rating == 'Low']['Review']
neg_dtm = vectorizer.fit_transform(neg_reviews)
neg_dtm_lsa = lsa.fit_transform(neg_dtm)
neg_dtm_lsa = Normalizer(copy=False).fit_transform(neg_dtm_lsa)
neg_lsa = pd.DataFrame(neg_dtm_lsa, index = neg_reviews, columns = ["component_1","component_2","component_3","component_4","component_5" ])

component 1: taste

In [75]:
neg_lsa.sort_values('component_1', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
I went to the Phoenix Hard Rock Cafe for lunch on a Friday. It wasn t very busy so I figured I would sit at the bar. The two servers/bartenders were super nice and friendly. So was management that I spoke with so I have nothing negative to say about the service. It was quick and friendly. But the menu! It sucks! It is way overpriced! And nothing I had was good. For $10 I got the worst plate of nachos I have ever had! Sure it was big and plenty to share and even more to throw out but it was awful. Glad I didn t pay another $4 for meat because it was so bad that even meat could not improve it! The beans had no flavor the jalapeno provided the only bit of flavor to the plate. The salsa was pure trash- more live crushed tomatoes and no flavor! You would think in the SW they could at least adapt and go with a better salsa! My other pick was potato skins I love the things. The guy next to me ordered them $9 for a half plate of the tiny-est potatoes I think anyone could skin! I think there were 6 halves but certainly not more than 8 halves all the size of a child s palm! I had a frozen drink called a Pickled Tink $6.75. Awful! Did not come completely mixed and barely pink. Tasted overwhelmingly of the coconut flavor. It was frozen and slushy and weak enough to feed to a kid. If I had one with me I would have let him/her have it seriously. It was my birthday as was maybe 4 other unfortunate souls to be eating there. They gave us each a Sundae with fudge on the whip cream and a small candle. They shouted at the patrons to to shoot Happy Birthday at each of us at separate times and clap. It was pretty weak by the third attempt to get participation. It was nice of my servers to surprise and acknowledge me. Though I was not looking for any handout or commotion. I had a few bites of the sundae and it was even awful! The ice cream was full of ice crystals! So over all I will have to disagree with others and say Applebee s food is much much better and not as expensive! To Applebee s I ll return but to the HRC this is my last visit. Years ago I went to the Phoenix HRC when it was on Camelback if I remember the street correctly. The building & bar was more impressive as was the decor and so was the food. I went on a birthday then too I wonder why they moved to this location? The entry is nothing eye catching or grand as it was then. In fact if the Light Rail driver had not announced it i would not have seen it as I had not seen it on previous trips to and pass the area. Before this stop at the HRC I had been a fan of the place for years. Had once spent many evenings at the Boston HRC when I lived in MA and had fond memories of the old Phoenix location as well. But just as the music we call Rock has deteriorated so has this conglomeration! So long fair well HRC I withdraw my support of you.,0.997757,0.035643,0.013533,0.032991,0.044041
Decent sushi I prefer to sit up at the bar counter. I went there for lunch the other day by myself and it was fast service but there were tons of regular types in there drinking and chatting up the staff. It made me wonder why they were just hanging out on a week day in the middle of the afternoon...but they are lucky I guess. Oh the food: I started with miso soup and edamame (both hard to mess up) they were good. I ordered a tempura shrimp roll...it was okay the end pieces had the parts you don t eat so there were only like four pieces. Overall it was okay and one of the only sushi places by my office (now that I work nearby) so I ll probably be back. I m open to recommendations...but I ve definitely been to plentyyyy of better sushi spots. Ohhh and the parking lot is a nightmare -- whether you re trying to run into Subway or Moto this shopping center is sheer chaos during lunch especially.,0.997702,-0.009090,-0.046845,-0.030324,-0.037325
Beats Burger King any day. I should ve manned up and just told my growling belly to quit being a whiny b

component 2: bad food quality

In [76]:
neg_lsa.sort_values('component_2', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Americanized Japanese food.,0.382128,0.878643,-0.096958,0.030165,0.267683
Grotesque food vulgar staff blech.,0.406268,0.877840,-0.086511,0.035543,0.235790
Over priced and mediocre food.,0.393311,0.875812,-0.096532,0.028194,0.261049
Excellent food as always,0.389897,0.875287,-0.088606,0.050488,0.267308
Wish the food was hotter.,0.394472,0.875021,-0.087446,0.033354,0.264521
Food is average. The hours are ridiculous! Its always closed????,0.409232,0.873912,-0.076899,0.026667,0.249365
It s Pei Wei. The service and food are always satisfactory.,0.436014,0.872509,0.012832,0.182656,0.122850
Decent food.,0.404465,0.870786,-0.108123,0.033767,0.255557
Incompetent staff. Medicore food. Not impressed enough to write much other than that you should go elsewhere.,0.427382,0.870447,-0.092096,0.038735,0.222902


component 3: bad service

In [77]:
neg_lsa.sort_values('component_3', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
It s too confusing to order here.Makes me get a rash.,0.482915,-0.048767,0.818747,0.300829,0.059757
Employees gathered in the back chatting so empty no only did they not notice me i had to call their phone on order to get them to come to the front,0.565244,-0.120123,0.784008,0.226621,0.006661
While I was eating a coach roach ran across my table! I immediately stopped eating and threw it in the trash and left disgusted.,0.546275,-0.050501,0.779597,0.289890,-0.085001
No!I said - NO!!!,0.442284,-0.299590,0.779129,0.319658,0.073534
Waitress took a long time to seat us. Then didn t give us an option on table choice (we didn t want to sit in the doorway) when I asked her to move she told me you can sit over there in such a rude tone. We decided to leave.,0.540576,-0.141385,0.765010,0.303465,-0.102254
Wildly uneven depending on who s in the kitchen and who s at the counter.,0.642815,0.010003,0.761896,0.064751,-0.044848
disappointing carnitas burrito. no kick in the dish despite being told it had spices. to compound that the seating is ridiculous. There are 2 fox restaurants side by side (blanco and modern burger). I ordered from blanco and assumed I would be allowed to seat in the dining area. As I proceeded to sit this lady comes running and rudely tells me you cannot sit here because you did not order a drink . Mind you I was siting on a crowded high table..... Not at the bar. When I politely old the server that I was waiting for myFood order from blanco she told me my order was to go (though I never asked for that). this despite there being at least 10 empty spots available. plain rude and terrible attitude. not getting my business ever again,0.568316,-0.129510,0.742571,0.329820,0.007134
They have changed their recipes and several items cannot be made vegan now. Be sure you ask before you order.,0.598604,-0.096650,0.742082,0.285638,0.007559
Walked in first before two other couples came in the door. Apparently they were regulars because when the hostess arrived she knew them by name and greeted them. After telling her we came in first she ignored us anyway and sat the regulars. After we finally got sat we never saw a single soul. She sat us went back over to the regulars to chat then dissapeared. We watched as everyone else got their order taken. We were ignored and waited about 20 minutes before getting up and leaving. I ll never go back again because of that. Sorry Nee House. We wanted to like you!,0.615604,-0.225340,0.737184,0.151225,0.062801


component 4: many think the food is pricy

In [78]:
neg_lsa.sort_values('component_4', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Solid. In and out in 45 minutes with a full stomach and not too much damage to the wallet. Nice staff and good service.,0.539128,-0.030728,-0.290244,0.752989,-0.239088
Not very good service. Orders came out wrong. Waited a long time for a glass for my beer. Waited too long for ketchup. Mistakes on check.,0.611523,-0.036162,0.064605,0.738841,-0.273261
Good service bbq was tastee. Overall happy!,0.467456,-0.066058,-0.494718,0.692860,-0.228735
Good sandwiches and quick service.,0.447463,-0.038852,-0.523953,0.680662,-0.245846
Not bad. Good wait staff. Quick service. She is carrying a gun. I would too if i worked on Van Buren.,0.542445,0.015040,-0.403809,0.667522,-0.311256
Appetizer platter was large and tasty.The galaktoboureko dessert was not good.Although the chef is apparently Greek he/she has made lots of compromises. Or did not pay attention to his Yaya.Service was super friendly.,0.613660,-0.130006,-0.381581,0.661071,-0.154600
So the burgers are good as most people on Yelp have posted but the experience could be better for a number of reasons. We had the $20 dinner special for 2 - where you get 2 burgers 1 order of fries for $20. So when I got the bill I was charged $21.25 and not $20 (this is not including tax) so I was thinking wait the special is advertised for $20 NOT $21.25! The server informed me that I was charged an extra $1.25 because I asked for onions on my burger! WTH!?? Are you serious?? The server never said that I would be charged extra for onions - so be careful when you order here because Zin likes to add deceptive charges - what a Sin Zin! Shysters.I reluctantly give 3 stars because the burger was good and the service was also pretty good. But Zin should include some food on the side with their burgers because when you order a burger you are getting it stand-alone no fries no veggies nutin.If you want to add extra on your burger be careful.,0.710411,-0.096757,-0.113392,0.659494,-0.195358
Here for conference. Convenient location attractive setting. Service was fine (not exceptional). Calamari and Greek fries a virtual sodium fest. Saganaki was good. Three humus platter a disappointment - they only had one flavor left. Over charged us we had to have the bill corrected.,0.551209,-0.147289,-0.459353,0.654763,-0.186424
Service was good even though they messed up my order. They did fix it and it was good. I had the salad with crab and shrimp. A little pricey though so not sure it s worth another visit or not..,0.579242,-0.225967,-0.417885,0.653313,-0.109417


In [79]:
neg_lsa.sort_values('component_5', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Chicken fried rice Yum. Friendly employees. Could have been cleaner.,0.314272,-0.210849,-0.238715,-0.334074,0.829570
The yougurt dipping sauce is delish!,0.307492,-0.334664,-0.237798,-0.286713,0.809133
Sigh... the search for anything remotely asian (authentic or fusion) continues. Coming from the Bay Area expectations are high. Yes we adjusted our expectations accordingly for the Phoenix area.The food had a ton of heat but lacked actual flavor.Napa Rolls: napa cabbage wrapped shrimp in a ponzu sauce the dish was cooked well but nothing stoodout eitherGinger Mint Duck Salad: douced with spicey chilli sauce the salad was drowning in a pool of liquid there is no duck taste as at allRed Thai Curry Chicken Wings: plain awful! terrible spicys used and horrible consistency the chicken was marinated in something super salty and then they mixed in a curry sauce that conflicted with the tasteWasabe Tofu: never had tofu with such a heavy batter on it it was not light and delicate beware of the sauce which is 5:1 ratio of wasabi to ponzu you can t even taste the tofu if you use the sauce,0.521498,-0.069653,-0.190221,-0.225993,0.797453
Took a feel bites of my Teriyaki Chicken and had to throw it out. Super sweet. Diabetics...beware!!,0.490743,-0.307515,-0.041139,-0.190566,0.791579
HORRIBLE!!!! I ordered Chicken Fried Rice General Tso s Chicken Orange Beef and egg rolls. Very poor cuts of meat. Terrible Flavor. Cabbage in egg rolls smelled and tasted rotten. I highly recommend going anywhere else.DO NOT EAT THERE!!!,0.459390,-0.305909,-0.085792,-0.266550,0.785475
CervicheFish tacosCombo - taco,0.409464,0.174739,-0.299977,-0.319582,0.780824
Avoid the lunch buffet. Excellent Chicken Tikka Masala.,0.428579,-0.215102,-0.266238,-0.310855,0.776233
I have tried the teriyaki chicken bowl and the orange chicken.... Both are bland... Think blender then Panda Express... But it s still feels healthier then Wendy s or the other junk at the airport,0.415324,-0.253351,-0.209025,-0.353854,0.770984
Ahi salad was poor....meat and dressing had no flavor. But the entrees were excellent - chicken enchiladas and carne adovada - yummy!,0.453774,-0.398010,-0.181454,-0.130077,0.765396
